In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
from datetime import datetime, timedelta
import json
import random

with open("credentials.json", "r") as credentials:
    credentials = json.load(credentials)

random.seed(time.time())
credentials = random.choice(credentials)
dialog = random.choice(credentials['dialogs'])

DATA_INICIO = (datetime.now()- timedelta(weeks=1)).strftime('%d%m%Y') 
HORA_INICIO = "0000"
DATA_FIM = datetime.now().strftime('%d%m%Y')
HORA_FIM = "2359"
CONVERSA = dialog
EMAIL = "pedro.nunes@omotor.com.br"
LOGIN = credentials['login']
SENHA = credentials['password']

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)  

""" chrome_options = Options()
chrome_service = Service(executable_path='./chromedriver') 
driver = webdriver.Chrome(service=chrome_service, options=chrome_options) """

try:
    driver.get('https://mones.omotor.com.br/')

    # Preencher o campo de Login
    login_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.NAME, "email"))
    )
    login_input.send_keys(LOGIN)

    # Preencher o Campo de Senha
    password_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.NAME, "password"))
    )
    password_input.send_keys(SENHA)

    # Clicar no botão de Entrar
    login_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.TAG_NAME, "button"))
    )
    login_button.click()

    # Trocar de Página
    time.sleep(1)
    WebDriverWait(driver, 5).until(
        EC.url_changes(driver.get('https://mones.omotor.com.br/recordings') )
    )
    
    # Preenchendo a data de início
    init_date_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "mat-input-5"))
    )
    init_date_input.send_keys(DATA_INICIO)

    # Preenchendo a hora de início
    init_hour_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "mat-input-2"))
    )
    init_hour_input.send_keys(HORA_INICIO)

    # Preenchendo a data de fim
    end_date_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "mat-input-6"))
    )
    end_date_input.send_keys(DATA_FIM)

    # Preenchendo a hora de fim
    end_hour_input = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "mat-input-3"))
    )
    end_hour_input.send_keys(HORA_FIM)

    # Selecionando a Lista
    select_element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "mat-input-4"))
    )
    select_element.send_keys(CONVERSA)

    select_element_child = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, f'//*[span[text()="{CONVERSA}"]]'))
    )
    select_element_child.click()

    # Clicar no botão de Pesquisa
    search_buton = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-describedby="cdk-describedby-message-ng-1-5"]'))
    )
    search_buton.click()

    # Clicar no botão de download
    download_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-describedby="cdk-describedby-message-ng-1-2"]'))
    )
    download_button.click()

    # Preencher o Formulário de Email
    input_element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "emailToSend"))
    )
    input_element.send_keys(EMAIL)

    # Clica no botão de enviar para o email
    send_email_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//button[.//span[@class="mdc-button__label" and text()=" Confirmar "]]'))
    )
    send_email_button.click()

    time.sleep(3)

except Exception as e:
    print("Erro",e)

finally:
    # Optionally close the browser
    driver.quit()


## Whatsapp Flood

In [ ]:
import os
import json
import time
from perguntas import perguntas
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

def save_response(response, file_path):
    data = []

    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='UTF-8') as json_file_read:
            data = json.load(json_file_read)
        
        data.append(response)
    
    elif type(response) != list: data.append(response)
    else: data = response

    with open(file_path, "w", encoding='UTF-8') as json_file_write:
        json.dump(data, json_file_write, indent=4, default=str, ensure_ascii=False)

def carregar_mensagens(file_path):
    with open(file_path, 'r', encoding='UTF-8') as file:
        return json.load(file)

def capturar_todas_respostas(driver:webdriver.Chrome):
    return driver.find_elements(By.XPATH, "//div[contains(@class, 'copyable-text') and contains(@class, 'selectable-text')]")

def capturar_resposta(driver, mensagens_iniciais):
    try:
        WebDriverWait(driver, 30).until(
            lambda driver: len(capturar_todas_respostas(driver)) > len(mensagens_iniciais)
        )
        novas_mensagens = capturar_todas_respostas(driver)
        return novas_mensagens[-1].text  
    except Exception as e:
        print(f"Erro ao capturar resposta: {e}")
        return None

def enviar_mensagens(perguntas):
    service = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=service)  
    
    driver.get("https://web.whatsapp.com")
    
    input("Escaneie o QR code do WhatsApp Web e pressione Enter quando estiver pronto.")
    
    # Abrir a conversa com o número de telefone
    numero = '11978092460'
    link = f"https://web.whatsapp.com/send?phone={numero}"
    driver.get(link)


    for pergunta in perguntas:

        try:
            input_element = WebDriverWait(driver, 60).until(
                EC.element_to_be_clickable((By.XPATH, '//div[@class="x1hx0egp x6ikm8r x1odjw0f x1k6rcq7 x6prxxf"]'))
            )
            input_element.send_keys(pergunta)

            botao_enviar = WebDriverWait(driver, 60).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@class="x1c4vz4f x2lah0s xdl72j9 xfect85 x1iy03kw x1lfpgzf"]'))
            )
            botao_enviar.click()

            time.sleep(1)
            
            mensagens_iniciais = capturar_todas_respostas(driver)
            
            data = []
            while True:
                resposta = capturar_resposta(driver, mensagens_iniciais)
                if resposta and resposta != pergunta:
                    print(f"Resposta recebida: {resposta}")
                    data.append({'question':pergunta,'answer':resposta})
                    save_response({'question':pergunta,'answer':resposta}, 'data.json')
                    break
                else:
                    print(f"Ainda nenhuma resposta capturada.")
                    time.sleep(1)

        except Exception as e:
            print(f"Erro ao enviar pergunta para {numero}: {e}/npergunta: {pergunta}")
    driver.quit()
    return data

# Executar o script
if __name__ == "__main__":
    print(enviar_mensagens(perguntas))


Erro ao capturar resposta: Message: 

Ainda nenhuma resposta capturada.
Erro ao capturar resposta: Message: 

Ainda nenhuma resposta capturada.


KeyboardInterrupt: 